How many items (and what items) is Country X the \#1 producter in for the most recent five years of data (2015-2020)? How many items is Country X the best producer (most efficient) producer in? 

In [1]:
import pandas as pd
import os

In [23]:
%run Methods.ipynb
%run TestProductionChange.ipynb

test_prepData (__main__.TestDataPrep) ... ok
test_withNA (__main__.TestDataPrep) ... ok
test_getItemsThatHaveElement (__main__.TestGetItemWithElements) ... ok
test_productionChange (__main__.TestItemToProdElement) ... ok
test_yieldPrepData (__main__.TestYieldDataPrep) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.092s

OK
test_prepData (__main__.TestDataPrep) ... ok
test_withNA (__main__.TestDataPrep) ... ok
test_getItemsThatHaveElement (__main__.TestGetItemWithElements) ... ok
test_productionChange (__main__.TestItemToProdElement) ... ok
test_yieldPrepData (__main__.TestYieldDataPrep) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.101s

OK
test_prepData (__main__.TestDataPrep) ... ok
test_withNA (__main__.TestDataPrep) ... ok
test_getItemsThatHaveElement (__main__.TestGetItemWithElements) ... ok
test_productionChange (__main__.TestItemToProdElement) ... ok
test_yieldPrepData (__main__.Te

In [3]:
pathToData = os.path.join(os.getcwd(), "Data", "Production_Crops_Livestock_E_All_Data.csv")
agData = pd.read_csv(pathToData, encoding="latin-1")
agData.head(2)

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F,Y2020,Y2020F
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,ha,NaN,NaN,NaN,...,19481.0,NaN,19793.0,NaN,20053.0,NaN,29203.0,NaN,22134.0,NaN
1,2,Afghanistan,221,"Almonds, with shell",5419,Yield,hg/ha,NaN,NaN,NaN,...,16859.0,Fc,13788.0,Fc,17161.0,Fc,13083.0,Fc,17759.0,Fc


# Clean the data

Drop the rows that are rollups - regions and China

In [14]:
agDataNoRegions = dropRegionRows(agData)
agDataClean = agDataNoRegions.drop(agDataNoRegions[agDataNoRegions['Area'] == 'China'].index)

Only want to see data for most recent 5 years 

In [15]:
fiveYearCols = createYearList(2015, 2020)
agDataFiveYears = agDataClean[["Area", "Item", "Element", "Unit"] + fiveYearCols]
agDataFiveYears.head(3)

,Area,Item,Element,Unit,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
0,Afghanistan,"Almonds, with shell",Area harvested,ha,14676.0,19481.0,19793.0,20053.0,29203.0,22134.0
1,Afghanistan,"Almonds, with shell",Yield,hg/ha,16521.0,16859.0,13788.0,17161.0,13083.0,17759.0
2,Afghanistan,"Almonds, with shell",Production,tonnes,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0


For each item, get the it's relevant production element, then subset the data to only be production data.

In [24]:
agDataProduction = subsetProductionData(agDataFiveYears)
agDataProduction.head(3)

,Area,Item,Element,Unit,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
2,Afghanistan,"Almonds, with shell",Production,tonnes,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0
5,Afghanistan,"Anise, badian, fennel, coriander",Production,tonnes,18000.0,17333.0,18944.0,18093.0,18123.0,18387.0
8,Afghanistan,Apples,Production,tonnes,89733.0,140903.0,170443.0,217192.0,250324.0,270857.0


For NAN values, fill with 0. 

In [31]:
agDataProduction.fillna(0, inplace=True)

C:\Users\14159\AppData\Local\Temp\ipykernel_27236\677681761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agDataProduction.fillna(0, inplace=True)


In [35]:
agDataProduction['Total'] = agDataProduction.loc[:, fiveYearCols].sum(axis = 1)
agDataProduction.head()

C:\Users\14159\AppData\Local\Temp\ipykernel_27236\638781829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agDataProduction['Total'] = agDataProduction.loc[:, fiveYearCols].sum(axis = 1)


,Area,Item,Element,Unit,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Total
2,Afghanistan,"Almonds, with shell",Production,tonnes,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0,196305.0
5,Afghanistan,"Anise, badian, fennel, coriander",Production,tonnes,18000.0,17333.0,18944.0,18093.0,18123.0,18387.0,108880.0
8,Afghanistan,Apples,Production,tonnes,89733.0,140903.0,170443.0,217192.0,250324.0,270857.0,1139452.0
11,Afghanistan,Apricots,Production,tonnes,87686.0,17894.0,131816.0,109086.0,129363.0,131788.0,607633.0
12,Afghanistan,Asses,Stocks,Head,1481000.0,1472100.0,1317000.0,1361141.0,1555247.0,1535435.0,8721923.0
